In [1]:
import pandas as pd
import numpy as np
import os
import gc
import datetime
import glob

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/Bruce_2019Q1_uploaded_groups'

In [2]:
file_id_groups=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/*.csv")

len(file_id_groups)

72

In [4]:
df_all_Q1_segments=pd.DataFrame()
for file in file_id_groups:
    df=pd.read_csv(file,dtype=str,usecols=['customer_id_hashed','segment'])
    df_all_Q1_segments=df_all_Q1_segments.append(df)
print(df_all_Q1_segments.shape)

(18654857, 2)


In [18]:
df_all_Q1_segments['customer_id_hashed'].nunique()

18654857

In [5]:
def recursive_file_gen(root_path):
    for root_path, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root_path,file)
            
list_2019_Q1_item_files=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
list_2019_Q1_item_files=[x for x in list_2019_Q1_item_files if "ily" in x]
df_2019_Q1_item_files=pd.DataFrame({"file_path":list_2019_Q1_item_files})
df_2019_Q1_item_files['week_end_dt']=df_2019_Q1_item_files['file_path'].apply(lambda x: x.split("MediaStorm_")[1][:10])
df_2019_Q1_item_files=df_2019_Q1_item_files[(df_2019_Q1_item_files['week_end_dt']<="2019-05-04") & (df_2019_Q1_item_files['week_end_dt']>="2019-02-16")]


extra_1_week=pd.DataFrame({"file_path":"/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20190209.txt",
                          "week_end_dt":"2019-02-09"},index=[0])

df_2019_Q1_item_files=df_2019_Q1_item_files.append(extra_1_week)
df_2019_Q1_item_files=df_2019_Q1_item_files.sort_values("week_end_dt").reset_index()
del df_2019_Q1_item_files['index']

In [14]:
df_all_sales_by_id_week=pd.DataFrame()
for file in df_2019_Q1_item_files['file_path'].tolist():
    df=pd.read_table(file,dtype=str,sep="|",
                    usecols=['location_id','customer_id_hashed','transaction_dt','transaction_id','item_transaction_amt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    df_sales=df.groupby(['customer_id_hashed','transaction_dt','location_id'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    
    df_trans=df[['customer_id_hashed','transaction_dt','location_id','transaction_id']].drop_duplicates()
    df_trans=df_trans.groupby(['customer_id_hashed','transaction_dt','location_id'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    
    df=pd.merge(df_sales,df_trans,on=['customer_id_hashed','transaction_dt','location_id'],how="outer")
    
    df['store_type']=np.where(df['location_id']=="6990","Online","Instore")
    df=df.groupby(['customer_id_hashed','transaction_dt','store_type'])['sales',"trans"].sum().reset_index()
    df_all_sales_by_id_week=df_all_sales_by_id_week.append(df)
    print(datetime.datetime.now(),file)
    
df_all_sales_by_id_week.shape

2019-07-31 13:40:21.805065 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20190209.txt
2019-07-31 13:43:52.552775 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-16/MediaStormDailySales20190219-113605-481.txt
2019-07-31 13:49:47.264970 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-23/MediaStormDailySales20190226-112611-940.txt
2019-07-31 13:55:38.727928 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-02/MediaStormDailySales20190305-112738-899.txt
2019-07-31 14:03:34.864230 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-09/MediaStormDailySales20190312-121205-996.txt
2019-07-31 14:13:26.368556 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-16/MediaStormDailySales20190319-112334-057.txt
2019-07-31 14:19:21.653047 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-23/MediaStormDailySales20190326-112740-903.txt
2019-07-31 14:24:04.619915 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-30/MediaStormDailySales20190402-112556-5

(17099863, 5)

In [16]:
df_all_sales_by_id_week['transaction_dt'].min()

'2019-02-03'

In [17]:
df_all_sales_by_id_week['transaction_dt'].max()

'2019-05-04'

In [30]:
df_all_sales_by_id_week.head(2)

,customer_id_hashed,transaction_dt,store_type,sales,trans
0,000009e0ff428cd05649d579c6b188ad41635b662f79f5...,2019-02-08,Instore,8.45,1
1,00002227290330cf6112d04470e512057ba4b50ab64c9f...,2019-02-07,Instore,4.05,1


In [34]:
df_all_sales_by_id=df_all_sales_by_id_week.groupby(['customer_id_hashed','store_type'])['sales','trans'].sum().reset_index()

df_all_sales_by_id_Ecom=df_all_sales_by_id[df_all_sales_by_id['store_type']=="Online"]
del df_all_sales_by_id_Ecom['store_type']
df_all_sales_by_id_Ecom=df_all_sales_by_id_Ecom.rename(columns={"sales":"sales_Online","trans":"trans_Online"})


df_all_sales_by_id_POS=df_all_sales_by_id[df_all_sales_by_id['store_type']=="Instore"]
del df_all_sales_by_id_POS['store_type']
df_all_sales_by_id_POS=df_all_sales_by_id_POS.rename(columns={"sales":"sales_Instore","trans":"trans_Instore"})

df_all_sales_by_id=pd.merge(df_all_sales_by_id_POS,df_all_sales_by_id_Ecom,on="customer_id_hashed",how="outer")
df_all_sales_by_id=df_all_sales_by_id.fillna(0)

df_all_sales_by_id=pd.merge(df_all_sales_by_id,df_all_Q1_segments,on="customer_id_hashed",how="outer")
df_all_sales_by_id['segment']=df_all_sales_by_id['segment'].fillna("not_defined")

df_all_sales_by_id.head(2)

,customer_id_hashed,sales_Instore,trans_Instore,sales_Online,trans_Online,segment
0,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,31.59,3.0,0.0,0.0,T_Quadrant I_P_H_2019Q1
1,000007eccbfdc528a745aea90d34404b4fae9fbad2525f...,15.44,1.0,0.0,0.0,not_defined


In [35]:
df_all_sales_by_id.shape

(20861451, 6)

In [36]:
df_all_sales_by_id['shoppers']=np.where(((df_all_sales_by_id['trans_Instore']>0) | (df_all_sales_by_id['trans_Online']>0)),1,0)
df_all_sales_by_id['id_count']=1

In [37]:
df_output_by_group=df_all_sales_by_id.groupby("segment")['id_count','shoppers','sales_Instore','trans_Instore','sales_Online','trans_Online'].sum().reset_index()
df_output_by_group.shape


(73, 7)

In [38]:
df_output_by_group.to_csv("./BL_2019Q1_liveramp_id_summary_by_group_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
